# 요구사항
1. 우리나라 대기업 국내외 관련기사->투자, 협약 맺음, 건설 내용잇음더 좋음
2. 중국기업 관련기사 국내외
3. 중국 기업이 우리나라에 투자하거나 우리나라 기업과 협력 관련기사->거의 안뜸
4. 중국 기업이 우리나라에 들어오는 기사->요즘은 알리,테무가 많이 뜨더라
이정도..?

# pip install & import & setting

In [440]:
# !pip install requests beautifulsoup4
# !pip install pandas
# !pip install openpyxl
# !pip install selenium
# !pip install webdriver_manager

Found existing installation: selenium 3.14.1
Uninstalling selenium-3.14.1:
  Would remove:
    /home/mimi99/.local/lib/python3.10/site-packages/selenium-3.14.1.dist-info/*
    /home/mimi99/.local/lib/python3.10/site-packages/selenium/*
Proceed (Y/n)?   Successfully uninstalled selenium-3.14.1
Defaulting to user installation because normal site-packages is not writeable
  Using cached selenium-3.14.1-py2.py3-none-any.whl (902 kB)


In [441]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

from datetime import datetime
import os

from urllib.parse import quote
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By

In [442]:
pd.set_option('display.max_colwidth', 100)

# Common Functions & Variables

In [443]:
current_time = datetime.now().strftime('%y%m%d%H%M%S')

In [444]:
def get_bs4_html(url: str, use_selenium = False):
    if use_selenium:
        driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
        driver.get(url)
        driver.implicitly_wait(10)
        page_source = driver.page_source
        driver.quit()
        soup = BeautifulSoup(page_source, 'html.parser')
        return soup
    
    else:
        response = requests.get(url)
        if not response.status_code == 200:
            print("Failed to retrieve the webpage")
            return

        html_content = response.content
        soup = BeautifulSoup(html_content, 'html.parser')
        return soup

# Naver Crawling

## `https://news.naver.com` setting

In [445]:
naver_news_url = "https://news.naver.com/"

## 속보 크롤링

In [446]:
# 속보
def newsflash_crawling(target_bs4: BeautifulSoup):
    newsflash_list = target_bs4.find('div', class_="cjs_nf_list _item_list").find_all('a')

    if not newsflash_list:
        print('속보를 찾을 수 없습니다.')
    else:
        newsflash_data = []
        for newsflash in newsflash_list:
            href = newsflash.get('href')
            title = newsflash.find('h4', class_="cn_title").text.strip()
            company_name = newsflash.find('div', class_="cn_name").text.strip()
            newsflash_data.append({'topic':'속보', 'title': title, 'company_name': company_name, 'link': href})
        
        df = pd.DataFrame(newsflash_data)
        return df

df_newsflash = newsflash_crawling(get_bs4_html(naver_news_url))
df_newsflash

,topic,title,company_name,link
0,속보,"'명룡대전' 인천 계양을, 민주당 이재명 당선 확실",중앙일보,https://n.news.naver.com/article/025/0003353339
1,속보,"산업장관 “중국관계 안정화, 동맹과 반도체장비 수출통제 공조”",매일경제,https://n.news.naver.com/article/009/0005286334
2,속보,"나경원, 경찰 출신 류삼영 꺾고 당선 확실",국민일보,https://n.news.naver.com/article/005/0001687714
3,속보,"부산 해운대을, 김미애 60.95% 득표 '당선 확실'",머니투데이,https://n.news.naver.com/article/008/0005023966
4,속보,"동작갑, 민주당 김병기 당선 확실…장진영 낙선",노컷뉴스,https://n.news.naver.com/article/079/0003884110
5,속보,"산업장관 ""中관계 안정화하며 동맹과 반도체장비 수출통제 공조""",연합뉴스,https://n.news.naver.com/article/001/0014623943
6,속보,"4·10총선 개표율 70%…현재 1위 민주 159곳, 국민의힘 92곳",강원일보,https://n.news.naver.com/article/087/0001037429
7,속보,이재명 '명룡대전' 승리...인천 계양을 당선 '확실',강원도민일보,https://n.news.naver.com/article/654/0000071403
8,속보,"'최대 격전지' 서울 동작을, 국민의힘 나경원 '당선 확실'",서울경제,https://n.news.naver.com/article/011/0004326022
9,속보,"이재명, 원희룡 누르고 인천 계양을 당선 확실",세계일보,https://n.news.naver.com/article/022/0003923104


## 섹션 크롤링

In [447]:
# TODO: 섹션 딕셔너리 만들기
def get_section_dictionary():
  
  return

# section map
section_dictionary = {
    101: {
        'main_section': "경제",
        'sub_section': {
          259: "금융",
          258: "증권",
            261: "산업/재계",
            771: "중기/벤처",
            260: "부동산",
            262: "글로벌 경제",
            310: "생활경제",
            263: "경제 일반"
        }
    },
    104: {
        'main_section': "세계",
        'sub_section': {
          231: "아시아/호주"
        }
    }
}

In [448]:
def section_crawling(target_bs4: BeautifulSoup, section_name: str):
    section_list = target_bs4.find('div', id="newsct").find_all('li')

    if not section_list:
        print(f'{section_name} 섹션을 찾을 수 없습니다.')
    else:
        section_data = []

        for section in section_list:
            content_html = section.find('div', class_="sa_text")
            if not content_html:
                continue
            href = content_html.find('a', class_="sa_text_title").get('href')
            title = content_html.find('strong', class_="sa_text_strong").text.strip()
            description = content_html.find('div', class_="sa_text_lede").text.strip()
            company_name = content_html.find('div', class_="sa_text_press").text.strip()
            
            section_data.append({'topic': section_name, 'title': title, 'company_name': company_name, "description": description,'link': href})
        
        df = pd.DataFrame(section_data)
        return df

In [449]:
all_section_df_list = []

for main_section_key, main_section_value in section_dictionary.items():
	main_section_html = get_bs4_html(f"https://news.naver.com/section/{main_section_key}")
	main_section_df = section_crawling(main_section_html, main_section_value['main_section'])

	if main_section_df is not None:
		print(f"{main_section_value['main_section']} 크롤링 성공")
		all_section_df_list.append(main_section_df)

	if main_section_value['sub_section']:
		for sub_section_key, sub_section_value in main_section_value['sub_section'].items():
			sub_section_html = get_bs4_html(f"https://news.naver.com/breakingnews/section/{main_section_key}/{sub_section_key}")
			sub_section_df = section_crawling(sub_section_html, sub_section_value)

			if main_section_df is not None:
				print(f"{main_section_value['main_section']} - {sub_section_value} 크롤링 성공")
				all_section_df_list.append(sub_section_df)

all_section_df = pd.concat(all_section_df_list, ignore_index=True)

경제 크롤링 성공
경제 - 금융 크롤링 성공
경제 - 증권 크롤링 성공
경제 - 산업/재계 크롤링 성공
경제 - 중기/벤처 크롤링 성공
경제 - 부동산 크롤링 성공
경제 - 글로벌 경제 크롤링 성공
경제 - 생활경제 크롤링 성공
경제 - 경제 일반 크롤링 성공
세계 크롤링 성공
세계 - 아시아/호주 크롤링 성공


## join df & to xlsx & to csv

In [450]:
total_news_df = pd.concat([df_newsflash, all_section_df])
total_news_df

,topic,title,company_name,link,description
0,속보,"'명룡대전' 인천 계양을, 민주당 이재명 당선 확실",중앙일보,https://n.news.naver.com/article/025/0003353339,NaN
1,속보,"산업장관 “중국관계 안정화, 동맹과 반도체장비 수출통제 공조”",매일경제,https://n.news.naver.com/article/009/0005286334,NaN
2,속보,"나경원, 경찰 출신 류삼영 꺾고 당선 확실",국민일보,https://n.news.naver.com/article/005/0001687714,NaN
3,속보,"부산 해운대을, 김미애 60.95% 득표 '당선 확실'",머니투데이,https://n.news.naver.com/article/008/0005023966,NaN
4,속보,"동작갑, 민주당 김병기 당선 확실…장진영 낙선",노컷뉴스,https://n.news.naver.com/article/079/0003884110,NaN
...,...,...,...,...,...
411,아시아/호주,"""美日, 방산정책조정위 발족…방위 무기 공동생산 논의"" 美당국자",뉴시스,https://n.news.naver.com/mnews/article/003/0012484362,미국과 일본이 10일(현지시각) 정상회담을 계기로 향후 방위 무기 공동 생산을 위한 협의체를 발족할 전망이다. 조 바이든 행정부 고위 당국자는 미일 정상회담을 앞두고 진행한...
412,아시아/호주,"[속보] 대만 강진 사망자 16명으로 늘어…""실종자 3명 수색 중""",서울경제,https://n.news.naver.com/mnews/article/011/0004325965,대만 강진 사망자 16명으로 늘어…실종자 3명 수색 중
413,아시아/호주,"외신들, 총선 출구조사 결과에 ""여당 참패…尹 레임덕 위기""",뉴스1,https://n.news.naver.com/mnews/article/421/0007472897,22대 총선의 출구조사 결과가 발표된 10일 외신들도 앞다퉈 보도를 쏟아냈다. 로이터통신과 AFP통신 등 외신들은 이날 범야권이 200석 안팎을 확보할 것이라는 지상파 3사...
414,아시아/호주,[속보] 대만 강진 사망자 16명으로 늘어…실종자 3명 수색 중,연합뉴스,https://n.news.naver.com/mnews/article/001/0014623326,


In [451]:
# to csv
total_news_df.to_csv(f'naver_{current_time}.csv', index=False)
print(f'naver_{current_time}.csv 파일 생성')

# to xlsx
file_name = 'naver.xlsx'

if os.path.isfile(file_name):
    with pd.ExcelWriter(file_name, mode='a') as writer:
        total_news_df.to_excel(writer, sheet_name=current_time, index=False)
        print(f'naver.xlsx 파일 {current_time} 시트 생성')
else:
    with pd.ExcelWriter(file_name) as writer:
        total_news_df.to_excel(writer, sheet_name=current_time, index=False)
        print(f'naver.xlsx 파일 생성')

naver_240411005443.csv 파일 생성
naver.xlsx 파일 240411005443 시트 생성


# investing.com Crawling

## `kr.investing.com` setting

In [452]:
investing_dot_com_url = "https://kr.investing.com/search/?tab=news&q="

## 키워드 검색

In [453]:
def investing_keyword_news_crawling(keyword: str):
  target_bs4 = get_bs4_html(f'{investing_dot_com_url}{quote(keyword)}', True)
  investing_news_list = target_bs4.find('div', class_="js-section-content largeTitle").find_all('div', class_="articleItem")

  if not investing_news_list:
      print('investing 뉴스를 찾을 수 없습니다.')
  else:
      investing_news_data = []
      for investing_news in investing_news_list:
        print(investing_news)
        #   href = investing_news.get('href')
        #   title = investing_news.find('h4', class_="cn_title").text.strip()
        #   company_name = investing_news.find('div', class_="cn_name").text.strip()
        #   investing_news_data.append({'topic':'속보', 'title': title, 'company_name': company_name, 'link': href})
      
      df = pd.DataFrame(investing_news_data)
      return df
keyword_news_df = investing_keyword_news_crawling('한국')
keyword_news_df

/bin/sh: 1: google-chrome: not found
/bin/sh: 1: google-chrome-stable: not found
/bin/sh: 1: google-chrome-beta: not found
/bin/sh: 1: google-chrome-dev: not found
/bin/sh: 1: google-chrome: not found
/bin/sh: 1: google-chrome-stable: not found
/bin/sh: 1: google-chrome-beta: not found
/bin/sh: 1: google-chrome-dev: not found


AttributeError: 'NoneType' object has no attribute 'split'